In [1]:
import os
import sys

os.chdir("C:/dataanalytics/python")
os.curdir

#Configure the environment . Set this up to the directory where spark is installed
if 'SPARK_HOME' not in os.environ:
    os.environ['SPARK_HOME'] = 'C:\\spark'
    
#create a variable for our root path
SPARK_HOME = os.environ['SPARK_HOME']

#Add the following paths to the system path. Please check your installation
#to make sure that these zip files actually exists. The names might change as
#versions change
sys.path.insert(0,os.path.join(SPARK_HOME,"python"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","mysql-connector-java-5.1.45.zip"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","py4j-0.10.6-src.zip"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","pyspark.zip"))
 
#Initialize a spark context
from pyspark import SparkContext
from pyspark import SparkConf

#optionally configure spark
conf = SparkConf()
conf.setAppName("Jeffwiz")

#Initalize spark context onl runs once
sc = SparkContext('local', conf=conf)




In [2]:
from pyspark.sql import SparkSession 
from pyspark.sql import DataFrame 
from pyspark.sql import Column 
from pyspark.sql import Row 
from pyspark.sql import GroupedData 
from pyspark.sql import DataFrameNaFunctions 
from pyspark.sql import DataFrameStatFunctions
from pyspark.sql import functions 
from pyspark.sql import types 
from pyspark.sql import Window 
from pyspark.sql import HiveContext
from pyspark.sql import SQLContext

In [3]:
spark = SparkSession.builder.master("local").appName("jeff").config(conf=conf).getOrCreate()

In [7]:
bankDF = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/bank") \
    .option("driver", "com.mysql.jdbc.Driver")\
    .option("dbtable", "account") \
    .option("user", "root") \
    .option("password", "Mog#67sag") \
    .load()

Py4JJavaError: An error occurred while calling o40.load.
: java.lang.ClassNotFoundException: com.mysql.jdbc.Driver
	at java.net.URLClassLoader.findClass(Unknown Source)
	at java.lang.ClassLoader.loadClass(Unknown Source)
	at java.lang.ClassLoader.loadClass(Unknown Source)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:45)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions$$anonfun$6.apply(JDBCOptions.scala:79)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions$$anonfun$6.apply(JDBCOptions.scala:79)
	at scala.Option.foreach(Option.scala:257)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:79)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:35)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:34)
	at org.apache.spark.sql.execution.datasources.DataSource.re

In [8]:
sc.stop

<bound method SparkContext.stop of <SparkContext master=local appName=Jeffwiz>>

In [ ]:
sc.stop()

In [ ]:
spark.read.format("jdbc").options(
url ="jdbc:mysql://localhost/mysql/bank",
driver="com.mysql.jdbc.Driver",
dbtable="account",
user="root",
password="Mog#67sag"
).load().take(10) 

In [4]:
import pandas as pd

In [5]:
l = [('Alice', 1)]

In [6]:
rdd = sc.parallelize(l)

In [7]:
df = spark.createDataFrame(rdd, ['name', 'age'])
df.collect()

[Row(name='Alice', age=1)]

In [8]:
spark.createDataFrame(df.toPandas()).collect()

[Row(name='Alice', age=1)]

In [9]:
from pyspark.sql.types import *

In [10]:
from pyspark.sql.functions import *

In [11]:
from pyspark.sql.types import IntegerType

In [12]:
sqlcontext = SQLContext(sc)

In [14]:
sqlcontext.registerFunction("stringlength", lambda x: len(x))

<function __main__.<lambda>>

In [15]:
sqlcontext.sql('select stringlength("test")').collect()

[Row(stringlength(test)='4')]

In [16]:
sqlcontext.udf.register("stringlength", lambda x: len(x))

<function __main__.<lambda>>

In [18]:
sqlcontext.sql('select stringlength("test")').collect()

[Row(stringlength(test)='4')]

In [19]:
df = sqlcontext.createDataFrame([(0.45,60.0, 40.1, "hello word"),
                                (2.3, 4.6, 56.5, "me too"),
                                (6.5, 5.5, 80.7, "hey there")], ["first", "second", "third", "fourth"])

In [20]:
df.show()

+-----+------+-----+----------+
|first|second|third|    fourth|
+-----+------+-----+----------+
| 0.45|  60.0| 40.1|hello word|
|  2.3|   4.6| 56.5|    me too|
|  6.5|   5.5| 80.7| hey there|
+-----+------+-----+----------+



In [21]:
sqlcontext.registerDataFrameAsTable(df, 'table1')

In [23]:
df2 = sqlcontext.sql("select first as f1, second as f2, third as f3, fourth as f4 from table1")

In [24]:
df2.collect()

[Row(f1=0.45, f2=60.0, f3=40.1, f4='hello word'),
 Row(f1=2.3, f2=4.6, f3=56.5, f4='me too'),
 Row(f1=6.5, f2=5.5, f3=80.7, f4='hey there')]

In [25]:
df2.toPandas()

,f1,f2,f3,f4
0,0.45,60.0,40.1,hello word
1,2.30,4.6,56.5,me too
2,6.50,5.5,80.7,hey there


In [28]:
df.explain()

== Physical Plan ==
Scan ExistingRDD[first#16,second#17,third#18,fourth#19]


In [29]:
df.explain

<bound method DataFrame.explain of DataFrame[first: double, second: double, third: double, fourth: string]>

In [31]:
df.printSchema()

root
 |-- first: double (nullable = true)
 |-- second: double (nullable = true)
 |-- third: double (nullable = true)
 |-- fourth: string (nullable = true)



In [33]:
df.head() #the first row

Row(first=0.45, second=60.0, third=40.1, fourth='hello word')

In [34]:
df.describe().show()

+-------+------------------+------------------+------------------+----------+
|summary|             first|            second|             third|    fourth|
+-------+------------------+------------------+------------------+----------+
|  count|                 3|                 3|                 3|         3|
|   mean|3.0833333333333335|23.366666666666664|              59.1|      null|
| stddev| 3.100134405688459|31.728588580857693|20.424495097798626|      null|
|    min|              0.45|               4.6|              40.1|hello word|
|    max|               6.5|              60.0|              80.7|    me too|
+-------+------------------+------------------+------------------+----------+

